In [12]:
%pip install ollama
%pip install python-docx
%pip install Pydantic  

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [13]:
# 1. Define the desired output structure using Pydantic
from pydantic import BaseModel, Field   

class Resume(BaseModel):
    """Data model for a parsed resume."""
    #Candidate name, Years of experience, Skill set, Education summary

    name: str = Field(description="full_name")
    skills: list[str] = Field(description="skills_summary")
    #experience: list[str] = Field(description="List of work experiences (job titles, companies, dates, responsibilities)")
    #education: list[str] = Field(description="List of educational background (degree, university, dates)")
    education_summary: str = Field(description="summary of education")
    years_of_experience: int = Field(description="total_years_of_experience")   

In [16]:
# Function to extract text from a .docx file
from docx import  Document  

def summarize_resume(file_path):
    # Load the resume document
    with open(file_path, 'rb') as file:
        document = Document(file)
        resume_text = '\n'.join([para.text for para in document.paragraphs])
    return resume_text

In [19]:
#Function to parse the resume using Ollama
import json
import ollama

def parse_resume_with_ollama(resume_text):
    prompt = f"""
    You are an expert AI assistant specializing in recruitment and talent acquisition.
    Your task is to parse the provided resume text and extract structured information.
    Ensure the output strictly follows the provided JSON schema.

    Resume Text:
    {resume_text}
    """

    # Call the Ollama model with JSON format
    response = ollama.chat(
        model="llama3", 
        messages=[{"role": "user", "content": prompt}],
        format="json", # Enforce JSON output
        options={
            'temperature': 0 # Aim for deterministic, accurate extraction
        }
    )

    # The response content will be a JSON string
    json_output = response['message']['content']
    
    return json.loads(json_output) 


In [20]:
#Main execution
   
# Replace 'your_resume.docx' with the path to your Word document
resume_file_path = './data/resume.docx'
    
# Check if the file exists before proceeding
try:
    with open(resume_file_path, 'rb'):
        pass
except FileNotFoundError:
    print(f"Error: {resume_file_path} not found. Please place your resume file in the correct path.")
    exit()

print(f"Extracting text from {resume_file_path}...")
#text = extract_text_from_docx(resume_file_path)
text = summarize_resume(resume_file_path)

if text:
    print("Parsing with Ollama...")
    parsed_resume = parse_resume_with_ollama(text)
    print("\n--- Parsed Resume (JSON Output) ---")
    json_str = json.dumps(parsed_resume, indent=4)
    print(json.dumps(parsed_resume, indent=4))
    try:
        with open("resume_summary.json", "w") as json_file:
            json_file.write(json_str)
            print("Resume summary saved to resume_summary.json")
    except Exception as e: 
            print(f"Error saving policy details: {e}")
else:
    print("Could not extract text from the document.")

Extracting text from ./data/resume.docx...
Parsing with Ollama...

--- Parsed Resume (JSON Output) ---
{
    "name": "Nirmala Kumar",
    "summary": "Transformational Product Leader and MIT-trained CTO with 20+ years of experience leading enterprise digital transformations and technical product delivery.",
    "corePillarsOfExpertise": [
        {
            "pillar": "Product Strategy & Innovation",
            "description": "Defining the 'North Star' for complex High-Tech products; aligning market needs with engineering feasibility."
        },
        {
            "pillar": "Architectural Strategy",
            "description": "Designing resilient, cloud-native frameworks for mission-critical enterprise applications (HL7/FHIR, HIPAA/HITECH)."
        },
        {
            "pillar": "Full-Cycle Delivery",
            "description": "Navigating the SDLC from algorithmic design to global deployment and CI/CD automation."
        },
        {
            "pillar": "High-Impact Lead

In [ ]:
#resume_text = summarize_resume("./data/resume.docx")
#print(f"resume_text: {resume_text}")


EXECUTIVE SUMMARY
Transformational Product Leader and MIT-trained CTO with 20+ years of experience leading enterprise digital transformations and technical product delivery. Specialized in turning complex clinical and technical goals into reliable, scalable, compliant systems. Proven track record in architectural strategy and platform modernization for large-scale organizations (AT&T, Microsoft, Deloitte) and public sector entities. Currently Founder & CEO of SpeakEQ, leveraging high-impact leadership to bridge the gap between engineering depth and C-suite objectives. Expert in reducing delivery risk while maximizing long-term value in high-stakes environments.
CORE PILLARS OF EXPERTISE
Product Strategy & Innovation: Defining the "North Star" for complex High-Tech products; aligning market needs with engineering feasibility.
Architectural Strategy: Designing resilient, cloud-native frameworks for mission-critical enterprise applications (HL7/FHIR, HIPAA/HITECH).
Full-Cycle Delivery: Na